# Statement of Purpose

__For this project, I wanted to make some sort of visualization dealing with public transportation in the greater Boston area. Why?__ 

* Firstly, I have been using publiic transport in Boston my whole life, and many of my most important memories involve it. Dozens of times throughout my childhood, my mom and I rode the trains into Boston, either to get bread from the Taiwanese bakeries in Chinatown, or simiply to walk around. These trips were simple - we weren't going on some extravagant journey to Disneyland - but they gave me a chance to connect with my mother and hold a special place in my heart as a result. 

* Secondly, visualizing the routes that trains take fulfils the basic idea of this project. Trains move from point A to point B, point B to point C, and so on, thereby connecting those points. Futhermore, some trains take paths that intersect other routes, turning individual routes into an interconnected web that spans an entire metropolitan area. TLDR, train routes are themselves a network. 

__What is my plan?__ 

* Naturally, my first step will be to import data (from the official MBTA database). Because I hope to visualize the public transportation routes as a set of points (representing individual stops), I will need a dataset providing the latitude and longitude of all train stops. To expand the scope of my project, I hope to add data about route reliability / timeliness as well as ridership into my visualization, and I will therefore need datasets with that sort of information. 

* My next step will be to wrangle my data into a form I can work with, after which I will use Altair to construct my visualization. I can use the latitude and longitude as my X and Y coordinates, and I can use either color or point size to depict reliability and ridership. 

__If you want to skip the code and go straight to the visualizations, go straight to the bottom of this notebook__
(Although I would prefer it if you at least glanced at the code, since I put a lot of work into it!). 
__Enjoy!__

# Imports / Libraries Used

In [237]:
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
from datetime import datetime

# Reliability Dataset

Explanation of columns: 
* service_d: Serviice date in yyyy-mm-dd (data spans midnight to midnight).

* gtfs_route_id: route ID 

* gtfs_route_short_name: A more human-readable name for the ID

* gtfs_route_long_name: A longer but still human-readable name for the ID

* gtfs_route_desc: GTFS attribute of route (is this commuter rail, rapid transit...)

* route_category: sub-category for a route (ex. "grey line" is a subcategory of rapid transport)

* mode_type: top level grouping (ex. bus)

* peak_off_ind: indicates wether data covers peak hours during the week

* metric_type: what attributes of performance are measured

* otp_numerator: number of observations where transport arrived reasonably on-time 

* otp_denominator: total number of observations or total ridership for a given service day. In any case, dividing otp_numberator by otp_denominator gives a reliability metric of on time / late (as well with cancelled_numerator). 

* cancelled_numerator: number of observations corresponding to cancelled service. 

These descriptions of the columns are brief and intended for my use while coding-- if you want a more detailed / comprehensive understanding of the dataset, I would look at this page: https://mbta-massdot.opendata.arcgis.com/datasets/MassDOT::mbta-bus-commuter-rail-rapid-transit-reliability/about

In [354]:
reldata = pd.read_csv("MBTA Reliability.csv")
reldata

,service_date,gtfs_route_id,gtfs_route_short_name,gtfs_route_long_name,gtfs_route_desc,route_category,mode_type,peak_offpeak_ind,metric_type,otp_numerator,otp_denominator,cancelled_numerator,ObjectId
0,2021/11/30 05:00:00+00,Red,NaN,NaN,Rapid Transit,Red Line,Rail,OFF_PEAK,Passenger Wait Time,116834.457868,131484.501852,NaN,1
1,2021/11/30 05:00:00+00,Red,NaN,NaN,Rapid Transit,Red Line,Rail,PEAK,Passenger Wait Time,103289.534378,117932.406159,NaN,2
2,2021/11/30 05:00:00+00,Orange,NaN,NaN,Rapid Transit,Orange Line,Rail,PEAK,Passenger Wait Time,76108.743163,81447.171476,NaN,3
3,2021/11/30 05:00:00+00,Orange,NaN,NaN,Rapid Transit,Orange Line,Rail,OFF_PEAK,Passenger Wait Time,89730.162237,97088.213286,NaN,4
4,2021/11/30 05:00:00+00,Green-E,NaN,NaN,Rapid Transit,Green Line,Rail,OFF_PEAK,Passenger Wait Time,17647.411728,22249.087595,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
683208,2015/08/01 04:00:00+00,104,NaN,NaN,Local Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,199.000000,392.000000,NaN,683209
683209,2015/08/01 04:00:00+00,101,NaN,NaN,Local Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,326.000000,446.000000,NaN,683210
683210,2015/08/01 04:00:00+00,100,NaN,NaN,Local Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,243.000000,329.000000,NaN,683211
683211,2015/08/01 04:00:00+00,10,NaN,NaN,Local Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,308.000000,500.000000,NaN,683212


## Data Modification

In [239]:
# we create extra columns describing reliability + cancellation metric (on time / cancelled over total # observations)
reliability_metric = reldata["otp_numerator"] / reldata["otp_denominator"]
cancellation_metric = reldata["cancelled_numerator"] / reldata["otp_denominator"]
reliability_metric 

0         0.888580
1         0.875837
2         0.934455
3         0.924213
4         0.793175
            ...   
683208    0.507653
683209    0.730942
683210    0.738602
683211    0.616000
683212    0.684825
Length: 683213, dtype: float64

In [240]:
reldata["cancelled_numerator"].isnull().value_counts() # there are actually no records for cancellation data, so cancellation_metric is redundant

True    683213
Name: cancelled_numerator, dtype: int64

In [241]:
# add the reloability metric
reldata["reliability_metric"] = reliability_metric
reldata

,service_date,gtfs_route_id,gtfs_route_short_name,gtfs_route_long_name,gtfs_route_desc,route_category,mode_type,peak_offpeak_ind,metric_type,otp_numerator,otp_denominator,cancelled_numerator,ObjectId,reliability_metric
0,2021/11/30 05:00:00+00,Red,NaN,NaN,Rapid Transit,Red Line,Rail,OFF_PEAK,Passenger Wait Time,116834.457868,131484.501852,NaN,1,0.888580
1,2021/11/30 05:00:00+00,Red,NaN,NaN,Rapid Transit,Red Line,Rail,PEAK,Passenger Wait Time,103289.534378,117932.406159,NaN,2,0.875837
2,2021/11/30 05:00:00+00,Orange,NaN,NaN,Rapid Transit,Orange Line,Rail,PEAK,Passenger Wait Time,76108.743163,81447.171476,NaN,3,0.934455
3,2021/11/30 05:00:00+00,Orange,NaN,NaN,Rapid Transit,Orange Line,Rail,OFF_PEAK,Passenger Wait Time,89730.162237,97088.213286,NaN,4,0.924213
4,2021/11/30 05:00:00+00,Green-E,NaN,NaN,Rapid Transit,Green Line,Rail,OFF_PEAK,Passenger Wait Time,17647.411728,22249.087595,NaN,5,0.793175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683208,2015/08/01 04:00:00+00,104,NaN,NaN,Local Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,199.000000,392.000000,NaN,683209,0.507653
683209,2015/08/01 04:00:00+00,101,NaN,NaN,Local Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,326.000000,446.000000,NaN,683210,0.730942
683210,2015/08/01 04:00:00+00,100,NaN,NaN,Local Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,243.000000,329.000000,NaN,683211,0.738602
683211,2015/08/01 04:00:00+00,10,NaN,NaN,Local Bus,Other Bus,Bus,OFF_PEAK,Headway / Schedule Adherence,308.000000,500.000000,NaN,683212,0.616000


## Group Data by Routes

In [242]:
# begin sorting the data- what types of routes are there and how many records are there for each?
reldata["route_category"].value_counts()

Other Bus      518794
Key Bus         56610
Silver Line     20990
Green Line      14636
Orange Line      3699
Blue Line        3698
Red Line         3697
Name: route_category, dtype: int64

In [243]:
reldata["gtfs_route_desc"].value_counts()

Local Bus                           412834
Commuter Rail                        60567
Key Bus Route (Frequent Service)     56610
Express Bus                          55132
Rapid Transit                        46720
Limited Service                      37694
Rail Replacement Bus                 13656
Name: gtfs_route_desc, dtype: int64

In [244]:
# commuter rail
cr_reldata = reldata[reldata["gtfs_route_desc"] == "Commuter Rail"]
cr_reldata

,service_date,gtfs_route_id,gtfs_route_short_name,gtfs_route_long_name,gtfs_route_desc,route_category,mode_type,peak_offpeak_ind,metric_type,otp_numerator,otp_denominator,cancelled_numerator,ObjectId,reliability_metric
12,2021/11/30 05:00:00+00,CR-Worcester,NaN,NaN,Commuter Rail,NaN,Commuter Rail,OFF_PEAK,Headway / Schedule Adherence,31.0,31.0,NaN,13,1.000000
13,2021/11/30 05:00:00+00,CR-Worcester,NaN,NaN,Commuter Rail,NaN,Commuter Rail,PEAK,Headway / Schedule Adherence,14.0,15.0,NaN,14,0.933333
20,2021/11/30 05:00:00+00,CR-Providence,NaN,NaN,Commuter Rail,NaN,Commuter Rail,PEAK,Headway / Schedule Adherence,11.0,12.0,NaN,21,0.916667
21,2021/11/30 05:00:00+00,CR-Providence,NaN,NaN,Commuter Rail,NaN,Commuter Rail,OFF_PEAK,Headway / Schedule Adherence,21.0,23.0,NaN,22,0.913043
22,2021/11/30 05:00:00+00,CR-Providence,NaN,NaN,Commuter Rail,NaN,Commuter Rail,PEAK,Headway / Schedule Adherence,9.0,9.0,NaN,23,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683019,2015/08/01 04:00:00+00,CR-Franklin,NaN,NaN,Commuter Rail,NaN,Commuter Rail,OFF_PEAK,Headway / Schedule Adherence,17.0,18.0,NaN,683020,0.944444
683020,2015/08/01 04:00:00+00,CR-Fitchburg,NaN,NaN,Commuter Rail,NaN,Commuter Rail,PEAK,Headway / Schedule Adherence,0.0,0.0,NaN,683021,NaN
683021,2015/08/01 04:00:00+00,CR-Fitchburg,NaN,NaN,Commuter Rail,NaN,Commuter Rail,OFF_PEAK,Headway / Schedule Adherence,1.0,1.0,NaN,683022,1.000000
683022,2015/08/01 04:00:00+00,CR-Fairmount,NaN,NaN,Commuter Rail,NaN,Commuter Rail,PEAK,Headway / Schedule Adherence,0.0,0.0,NaN,683023,NaN


In [245]:
# rapid transit
rt_reldata = reldata[reldata["gtfs_route_desc"] == "Rapid Transit"]
rt_reldata

,service_date,gtfs_route_id,gtfs_route_short_name,gtfs_route_long_name,gtfs_route_desc,route_category,mode_type,peak_offpeak_ind,metric_type,otp_numerator,otp_denominator,cancelled_numerator,ObjectId,reliability_metric
0,2021/11/30 05:00:00+00,Red,NaN,NaN,Rapid Transit,Red Line,Rail,OFF_PEAK,Passenger Wait Time,116834.457868,131484.501852,NaN,1,0.888580
1,2021/11/30 05:00:00+00,Red,NaN,NaN,Rapid Transit,Red Line,Rail,PEAK,Passenger Wait Time,103289.534378,117932.406159,NaN,2,0.875837
2,2021/11/30 05:00:00+00,Orange,NaN,NaN,Rapid Transit,Orange Line,Rail,PEAK,Passenger Wait Time,76108.743163,81447.171476,NaN,3,0.934455
3,2021/11/30 05:00:00+00,Orange,NaN,NaN,Rapid Transit,Orange Line,Rail,OFF_PEAK,Passenger Wait Time,89730.162237,97088.213286,NaN,4,0.924213
4,2021/11/30 05:00:00+00,Green-E,NaN,NaN,Rapid Transit,Green Line,Rail,OFF_PEAK,Passenger Wait Time,17647.411728,22249.087595,NaN,5,0.793175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683084,2015/08/02 04:00:00+00,751,NaN,NaN,Rapid Transit,Silver Line,Bus,OFF_PEAK,Headway / Schedule Adherence,1427.000000,1528.000000,NaN,683085,0.933901
683085,2015/08/02 04:00:00+00,749,NaN,NaN,Rapid Transit,Silver Line,Bus,OFF_PEAK,Headway / Schedule Adherence,2230.000000,2712.000000,NaN,683086,0.822271
683086,2015/08/02 04:00:00+00,746,NaN,NaN,Rapid Transit,Silver Line,Bus,OFF_PEAK,Headway / Schedule Adherence,56.000000,121.000000,NaN,683087,0.462810
683087,2015/08/02 04:00:00+00,742,NaN,NaN,Rapid Transit,Silver Line,Bus,OFF_PEAK,Headway / Schedule Adherence,506.000000,752.000000,NaN,683088,0.672872


# Rapid Transit Stops Dataset

Description (simplified)

> This dataset gives the name, longitude, and latitude of all rapid transit stops. 

In [246]:
rtstopsdata = pd.read_csv("MBTA RT Stops.csv")
rtstopsdata

,X,Y,OBJECTID,stop_id,stop_code,stop_name,stop_desc,platform_code,platform_name,stop_lat,...,Sidewalk_Condition,Sidewalk_Material,Current_Shelter,Routes,Municipality_1,Neighborhood,created_user,created_date,last_edited_user,last_edited_date
0,-71.142483,42.395428,618611,place-alfcl,NaN,Alewife,NaN,NaN,NaN,42.395428,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00
1,-71.137955,42.348701,618622,place-alsgr,NaN,Allston Street,NaN,NaN,NaN,42.348701,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00
2,-71.114748,42.350992,618625,place-amory,NaN,Amory Street,NaN,NaN,NaN,42.350992,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00
3,-71.057655,42.330154,618632,place-andrw,NaN,Andrew,NaN,NaN,NaN,42.330154,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00
4,-71.030395,42.374262,618640,place-aport,NaN,Airport,NaN,NaN,NaN,42.374262,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,-71.022865,42.379640,619387,place-wimnl,NaN,Wood Island,NaN,NaN,NaN,42.379640,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00
117,-71.020337,42.266514,619393,place-wlsta,NaN,Wollaston,NaN,NaN,NaN,42.266514,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00
118,-70.991648,42.413420,619400,place-wondl,NaN,Wonderland,NaN,NaN,NaN,42.413420,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00
119,-71.243362,42.332902,619409,place-woodl,NaN,Woodland,NaN,NaN,NaN,42.332902,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00


#  Commuter Rail Stops Dataset

Description (simplified)

> This dataset gives the name, longitude, and latitude of all commuter rail stops. 

In [247]:
crstopsdata = pd.read_csv("MBTA CR Stops.csv")
crstopsdata

,X,Y,OBJECTID,stop_id,stop_code,stop_name,stop_desc,platform_code,platform_name,stop_lat,...,Sidewalk_Condition,Sidewalk_Material,Current_Shelter,Routes,Municipality_1,Neighborhood,created_user,created_date,last_edited_user,last_edited_date
0,-70.714722,41.758333,618159,CM-0493-S,NaN,Wareham Village,Wareham Village - CapeFLYER - Track 1 (All Tra...,1.0,Track 1 (All Trains),41.758333,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00
1,-70.616226,41.744805,618161,CM-0547-S,NaN,Buzzards Bay,Buzzards Bay - CapeFLYER - Track 1 (All Trains),1.0,Track 1 (All Trains),41.744805,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00
2,-70.588772,41.746497,618163,CM-0564-S,NaN,Bourne,Bourne - CapeFLYER - Track 1 (All Trains),1.0,Track 1 (All Trains),41.746497,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00
3,-70.276583,41.660225,618165,CM-0790-S,NaN,Hyannis,Hyannis - CapeFLYER - Track 1 (All Trains),1.0,Track 1 (All Trains),41.660225,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00
4,-71.133246,42.238405,618167,DB-0095,NaN,Readville,Readville - Commuter Rail,NaN,Commuter Rail,42.238405,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,-71.054882,42.351527,619297,NEC-2287-09,NaN,South Station,South Station - Commuter Rail - Track 9,9.0,Commuter Rail - Track 9,42.351527,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00
341,-71.054914,42.351081,619298,NEC-2287-10,NaN,South Station,South Station - Commuter Rail - Track 10,10.0,Commuter Rail - Track 10,42.351081,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00
342,-71.054914,42.351081,619299,NEC-2287-11,NaN,South Station,South Station - Commuter Rail - Track 11,11.0,Commuter Rail - Track 11,42.351081,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00
343,-71.054938,42.350778,619300,NEC-2287-12,NaN,South Station,South Station - Commuter Rail - Track 12,12.0,Commuter Rail - Track 12,42.350778,...,NaN,NaN,NaN,NaN,NaN,NaN,DOT_ADMIN,2021/12/07 11:33:56+00,DOT_ADMIN,2021/12/07 11:33:56+00


# Commuter Rail Ridership

Description (simple): 

> This dataset includes ridership (how many people get on, how many people exit) for all commuter rail stops. More importantly for my purposes, it details which commuter rail route each stop belongs to (this will help immensely with plotting later). 

Long Description of Columns (from data website):

* mode	GTFS-compatible mode of transportation for which ridership should be returned.
* season	Season and year for which ridership should be returned.	
* route_id	GTFS-compatible route for which ridership should be returned.	
* route_name	Description of route.	
* direction_id	GTFS-compatible direction for which ridership should be returned. A 0 indicates "inbound" and 1 indicates "outbound."	
* day_type_id	Shorthand for day identifier.	
* day_type_name	Text description of the id. Weekday, Saturday, or Sunday; holidays are excluded from the data.	
* stop_time	Time at which the vehicle arrived at the stop. Represented in 24-hour time.
* stop_name	GTFS-compatible stop name for which ridership should be returned.	
* stop_id	GTFS-compatible stop for which ridership should be returned.	
* stop_sequence	The ordinal identifier of the stop on the particular trip.	
* average_ons	The count of passengers getting on vehicles on a typical weekday.	
* average_offs	The count of passengers getting on vehicles on a typical weekday.	
* __average_load	The average number of passengers on the vehicle upon leaving a stop, summed by the aggregated fields.__

In [248]:
cr_ridership_data = pd.read_csv("MBTA CR Ridership.csv")
cr_ridership_data['route_id'].value_counts()

CR-Newburyport      1499
CR-Worcester        1233
CR-Providence       1185
CR-Fitchburg        1076
CR-Haverhill         945
CR-Franklin          912
CR-Lowell            802
CR-Needham           724
CR-Fairmount         465
CR-Greenbush         431
CR-Kingston          418
CR-Middleborough     404
Name: route_id, dtype: int64

In [249]:
# modify so that data is organized by weekday/weekend and stop, and displays 
cr_ridership2 = cr_ridership_data.groupby(["route_id", "stop_name", "day_type_name"])[["average_ons", "average_offs"]].mean()

cr_ridership2 # WOW this looks nice

average_ons  average_offs
route_id     stop_name           day_type_name                           
CR-Fairmount Fairmount           weekday          10.075758      9.850746
             Four Corners/Geneva weekday           6.282051      6.052632
             Morton Street       weekday           5.000000      4.582090
             Newmarket           weekday           1.875000      2.200000
             Readville           weekday           9.416667     10.162162
...                                                     ...           ...
CR-Worcester West Natick         weekday          19.775281     18.133333
             West Newton         weekday           9.127660     10.468085
             Westborough         weekday          19.790323     17.774194
             Worcester           weekday          65.088235     65.718750
             Yawkey              weekday          21.625000     24.458333

[161 rows x 2 columns]

# Rapid Transit Ridership

Description: 

> This dataset includes ridership (how many people get on, how many people exit) for all rapid transit stops. More importantly for my purposes, it details which rapid transit route each stop belongs to (this will help immensely with plotting later). 

Long Description (from data website): 

* mode	GTFS-compatible mode of transportation for which ridership should be returned.	
* season	Season and year for which ridership should be returned.
* route_id	GTFS-compatible route for which ridership should be returned.
* route_name	Description of route.	
* direction_id	GTFS-compatible direction for which ridership should be returned.	
* day_type_id	Shorthand for day identifier.	
* day_type_name	Text description of the id. Weekday, Saturday, or Sunday; holidays are excluded from the data.
* time_period_id	Shorthand for time period identifier, per our Service Delivery Policy.
* time_period_name	Aggregated periods of varying lengths to represent different levels of service provided.
* stop_name	GTFS-compatible stop name for which ridership should be returned.
* stop_id	GTFS-compatible stop for which ridership should be returned.	
* total_ons	The count of passengers boarding vehicles, summed by the aggregated fields.	
* total_offs	The count of passengers alighting vehicles, summed by the aggregated fields.	
* number_service_days	Number of non-holiday service days in the season based on day type.
* average_ons	Represents the ons on a typical service day within the aggregated fields. Calculated by total_ons/number_service_days.	
* average_offs	Represents the offs on a typical service day within the aggregated fields. Calculated by total_offs/number_service_days. 	
* __average_flow	The total number of passengers traveling through the rail system between stations within the aggregated time period, averaged to represent the typical weekday, saturday, or sunday.__	

In [250]:
rt_ridership_data = pd.read_csv("MBTA RT Ridership.csv")
rt_ridership_data["route_id"].value_counts() # hmmmm no silver line data... unfortunate

Green     4356
Red       1452
Orange    1320
Blue       792
Name: route_id, dtype: int64

In [251]:
# modify so that data is organized by weekday/weekend and stop, and displays 
rt_ridership2 = rt_ridership_data.groupby(["route_id", "stop_name", "day_type_name"])[["average_ons", "average_offs"]].mean()

rt_ridership2 # WOW this looks nice

average_ons  average_offs
route_id stop_name     day_type_name                           
Blue     Airport       saturday       2899.166667   2604.500000
                       sunday         2654.000000   2125.833333
                       weekday         443.703704    391.203704
         Aquarium      saturday       2144.833333   1876.833333
                       sunday         1715.166667   1436.166667
...                                           ...           ...
Red      South Station sunday         4373.833333   2799.000000
                       weekday        1597.851852   1432.370370
         Wollaston     saturday        540.000000    597.500000
                       sunday          457.000000    506.000000
                       weekday         141.925926    154.666667

[357 rows x 2 columns]

# Assembling Data

We need to concatonate parts of each of the above datasets in order to accomplish what we want. ___What do we want, exactly___? 

* The visualization should primarily display all routes as lines joining together points that represent the ride stops. Stops are plotted according to latitude and longitude, and their color is determined by the route they are a part of. 

* There will be two "overlays": one for total route ridership and one for route reliability (how on-time is transportation?). Each overlay will be represented by its own graph (so that no user input is required). 

* Because there are two datasets for transport type (rapid transit and commuter rail), we will need to split the two already-existing overlays into four, where the four overlays

* The overlays will, finally, need to be split once more into weekday and weekend groups. 

After this laborious process, __we will be ready to visualize the data__. 

### CR-Ridership

In [252]:
cr_ridership_data# this is the data we need to modify. All we need to do is add the lat and long. 

,mode,season,route_id,route_name,direction_id,day_type_id,day_type_name,stop_time,stop_name,stop_id,stopsequence,average_ons,average_offs,average_load,ObjectId
0,2,Spring 2018,CR-Fairmount,Fairmount Line,0,day_type_01,weekday,2018/01/01 10:39:00+00,Readville,place-DB-0095,1,5.0,0.0,5.0,1
1,2,Spring 2018,CR-Fairmount,Fairmount Line,0,day_type_01,weekday,2018/01/01 10:42:00+00,Fairmount,place-DB-2205,2,7.0,0.0,12.0,2
2,2,Spring 2018,CR-Fairmount,Fairmount Line,0,day_type_01,weekday,2018/01/01 10:47:00+00,Morton Street,place-DB-2230,3,9.0,1.0,20.0,3
3,2,Spring 2018,CR-Fairmount,Fairmount Line,0,day_type_01,weekday,2018/01/01 10:50:00+00,Talbot Avenue,place-DB-2240,4,5.0,1.0,24.0,4
4,2,Spring 2018,CR-Fairmount,Fairmount Line,0,day_type_01,weekday,2018/01/01 10:53:00+00,Four Corners/Geneva,place-DB-2249,5,6.0,1.0,29.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10089,2,Spring 2012,CR-Worcester,Framingham/Worcester Line,1,day_type_01,weekday,2018/01/02 05:00:00+00,Wellesley Hills,place-WML-0135,8,0.0,1.0,27.0,10090
10090,2,Spring 2012,CR-Worcester,Framingham/Worcester Line,1,day_type_01,weekday,2018/01/02 05:04:00+00,Wellesley Square,place-WML-0147,9,0.0,1.0,26.0,10091
10091,2,Spring 2012,CR-Worcester,Framingham/Worcester Line,1,day_type_01,weekday,2018/01/02 05:10:00+00,Natick Center,place-WML-0177,10,0.0,5.0,21.0,10092
10092,2,Spring 2012,CR-Worcester,Framingham/Worcester Line,1,day_type_01,weekday,2018/01/02 05:16:00+00,West Natick,place-WML-0199,11,0.0,3.0,18.0,10093


In [253]:
cr_ridership_data["day_type_name"].unique() # only weekdays are shown. 

array(['weekday'], dtype=object)

In [254]:
# make a list of lat and long values to be concatenated (???) with the ridership dataset
cr_latlong = []
nonstops = []
for stop in cr_ridership_data["stop_name"]:

    if stop in crstopsdata["stop_name"].values: # some of the ridership stops are not in the stops dataset... 

        cr_latlong.append(
            [
                crstopsdata[crstopsdata["stop_name"] == stop]["X"].values[0],
                crstopsdata[crstopsdata["stop_name"] == stop]["Y"].values[0],
                crstopsdata[crstopsdata["stop_name"] == stop]["stop_name"].values[0],
                cr_ridership_data[cr_ridership_data["stop_name"] == stop]["route_id"].values[0],
                cr_ridership_data[cr_ridership_data["stop_name"] == stop]["day_type_name"].values[0]
            ]
            )

    else:
        nonstops.append(stop) # the ridership stops not included in the stops dataset: what are they? 

cr_latlong = pd.DataFrame(cr_latlong, columns=["X", "Y", "stop_name","route_id", "daytype"])
    

In [255]:
cr_latlong # looks great! 

,X,Y,stop_name,route_id,daytype
0,-71.133246,42.238405,Readville,CR-Fairmount,weekday
1,-71.119270,42.253638,Fairmount,CR-Fairmount,weekday
2,-71.085475,42.280994,Morton Street,CR-Fairmount,weekday
3,-71.078140,42.292246,Talbot Avenue,CR-Fairmount,weekday
4,-71.076833,42.305037,Four Corners/Geneva,CR-Fairmount,weekday
...,...,...,...,...,...
9805,-71.277044,42.310370,Wellesley Hills,CR-Worcester,weekday
9806,-71.294173,42.297526,Wellesley Square,CR-Worcester,weekday
9807,-71.347133,42.285719,Natick Center,CR-Worcester,weekday
9808,-71.391797,42.283064,West Natick,CR-Worcester,weekday


In [256]:
# now, we modify the ridership dataset so that it does not include stops which aren't listed in the stops dataset,
cr_ridership_f = cr_ridership_data[~cr_ridership_data["stop_name"].isin(pd.Series(nonstops).unique())] # nice
cr_ridership_f.index = cr_latlong.index

# we can then append the lat and lon: 
cr_ridership_f = pd.concat([cr_ridership_f,cr_latlong[["X", "Y"]]], axis=1)
cr_ridership_f


,mode,season,route_id,route_name,direction_id,day_type_id,day_type_name,stop_time,stop_name,stop_id,stopsequence,average_ons,average_offs,average_load,ObjectId,X,Y
0,2,Spring 2018,CR-Fairmount,Fairmount Line,0,day_type_01,weekday,2018/01/01 10:39:00+00,Readville,place-DB-0095,1,5.0,0.0,5.0,1,-71.133246,42.238405
1,2,Spring 2018,CR-Fairmount,Fairmount Line,0,day_type_01,weekday,2018/01/01 10:42:00+00,Fairmount,place-DB-2205,2,7.0,0.0,12.0,2,-71.119270,42.253638
2,2,Spring 2018,CR-Fairmount,Fairmount Line,0,day_type_01,weekday,2018/01/01 10:47:00+00,Morton Street,place-DB-2230,3,9.0,1.0,20.0,3,-71.085475,42.280994
3,2,Spring 2018,CR-Fairmount,Fairmount Line,0,day_type_01,weekday,2018/01/01 10:50:00+00,Talbot Avenue,place-DB-2240,4,5.0,1.0,24.0,4,-71.078140,42.292246
4,2,Spring 2018,CR-Fairmount,Fairmount Line,0,day_type_01,weekday,2018/01/01 10:53:00+00,Four Corners/Geneva,place-DB-2249,5,6.0,1.0,29.0,5,-71.076833,42.305037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9805,2,Spring 2012,CR-Worcester,Framingham/Worcester Line,1,day_type_01,weekday,2018/01/02 05:00:00+00,Wellesley Hills,place-WML-0135,8,0.0,1.0,27.0,10090,-71.277044,42.310370
9806,2,Spring 2012,CR-Worcester,Framingham/Worcester Line,1,day_type_01,weekday,2018/01/02 05:04:00+00,Wellesley Square,place-WML-0147,9,0.0,1.0,26.0,10091,-71.294173,42.297526
9807,2,Spring 2012,CR-Worcester,Framingham/Worcester Line,1,day_type_01,weekday,2018/01/02 05:10:00+00,Natick Center,place-WML-0177,10,0.0,5.0,21.0,10092,-71.347133,42.285719
9808,2,Spring 2012,CR-Worcester,Framingham/Worcester Line,1,day_type_01,weekday,2018/01/02 05:16:00+00,West Natick,place-WML-0199,11,0.0,3.0,18.0,10093,-71.391797,42.283064


In [257]:
cr_rid_f = [] # better organization: form a data table of [Lat, Long, Route name, and average daily load for all routes.]

for x in cr_latlong["X"].unique():

    for y in cr_latlong[cr_latlong["X"]==x]["Y"].unique():

        for route in cr_latlong[cr_latlong["X"]==x]["route_id"].unique():

            for daytype in ["weekday", "weekend"]:

                cr_rid_f.append(
                    [
                    x,
                    y,
                    cr_latlong[cr_latlong["X"] == x]["stop_name"].values[0],
                    route,
                    daytype,
                    cr_ridership_f[(cr_ridership_f["route_id"]==route) & (cr_ridership_f["day_type_name"]==daytype)]["average_load"].mean()
                    ]
                    )
            

cr_rid_f = pd.DataFrame(cr_rid_f)
cr_rid_f.columns = ["X", "Y","Stop", "Route", "day", "average_load"]
cr_rid_f 

,X,Y,Stop,Route,day,average_load
0,-71.133246,42.238405,Readville,CR-Fairmount,weekday,35.987277
1,-71.133246,42.238405,Readville,CR-Fairmount,weekend,NaN
2,-71.119270,42.253638,Fairmount,CR-Fairmount,weekday,35.987277
3,-71.119270,42.253638,Fairmount,CR-Fairmount,weekend,NaN
4,-71.085475,42.280994,Morton Street,CR-Fairmount,weekday,35.987277
...,...,...,...,...,...,...
265,-71.647076,42.269644,Westborough,CR-Worcester,weekend,NaN
266,-71.139883,42.357293,Boston Landing,CR-Worcester,weekday,178.115996
267,-71.139883,42.357293,Boston Landing,CR-Worcester,weekend,NaN
268,-71.794888,42.261461,Worcester,CR-Worcester,weekday,178.115996


### CR-Reliability

In [258]:
cr_reldata # we're going to need to do something about that "service_date" column... 

,service_date,gtfs_route_id,gtfs_route_short_name,gtfs_route_long_name,gtfs_route_desc,route_category,mode_type,peak_offpeak_ind,metric_type,otp_numerator,otp_denominator,cancelled_numerator,ObjectId,reliability_metric
12,2021/11/30 05:00:00+00,CR-Worcester,NaN,NaN,Commuter Rail,NaN,Commuter Rail,OFF_PEAK,Headway / Schedule Adherence,31.0,31.0,NaN,13,1.000000
13,2021/11/30 05:00:00+00,CR-Worcester,NaN,NaN,Commuter Rail,NaN,Commuter Rail,PEAK,Headway / Schedule Adherence,14.0,15.0,NaN,14,0.933333
20,2021/11/30 05:00:00+00,CR-Providence,NaN,NaN,Commuter Rail,NaN,Commuter Rail,PEAK,Headway / Schedule Adherence,11.0,12.0,NaN,21,0.916667
21,2021/11/30 05:00:00+00,CR-Providence,NaN,NaN,Commuter Rail,NaN,Commuter Rail,OFF_PEAK,Headway / Schedule Adherence,21.0,23.0,NaN,22,0.913043
22,2021/11/30 05:00:00+00,CR-Providence,NaN,NaN,Commuter Rail,NaN,Commuter Rail,PEAK,Headway / Schedule Adherence,9.0,9.0,NaN,23,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683019,2015/08/01 04:00:00+00,CR-Franklin,NaN,NaN,Commuter Rail,NaN,Commuter Rail,OFF_PEAK,Headway / Schedule Adherence,17.0,18.0,NaN,683020,0.944444
683020,2015/08/01 04:00:00+00,CR-Fitchburg,NaN,NaN,Commuter Rail,NaN,Commuter Rail,PEAK,Headway / Schedule Adherence,0.0,0.0,NaN,683021,NaN
683021,2015/08/01 04:00:00+00,CR-Fitchburg,NaN,NaN,Commuter Rail,NaN,Commuter Rail,OFF_PEAK,Headway / Schedule Adherence,1.0,1.0,NaN,683022,1.000000
683022,2015/08/01 04:00:00+00,CR-Fairmount,NaN,NaN,Commuter Rail,NaN,Commuter Rail,PEAK,Headway / Schedule Adherence,0.0,0.0,NaN,683023,NaN


In [259]:
# we can use datetime.strptime to obtain the weekday of the date shown.
# here is an example, where the weekday of the date in the first entry in the dataset is shown: (that sentence was horrible, I'm so sorry)

datetime.strptime(cr_reldata["service_date"][12][:10], '%Y/%m/%d').weekday()

1

In [260]:
# we apply it to all the dates,
cr_servicedate = cr_reldata["service_date"].apply(lambda date: datetime.strptime(date[:10], '%Y/%m/%d').weekday())

# then we convert to weekend / weekday (monday is 0 and sunday is 6):
cr_servicedate = cr_servicedate.apply(lambda date: "weekday" if date < 5 else "weekend")
cr_servicedate.name = "mod_service_date"
cr_servicedate # wa la

12        weekday
13        weekday
20        weekday
21        weekday
22        weekday
           ...   
683019    weekend
683020    weekend
683021    weekend
683022    weekend
683023    weekend
Name: mod_service_date, Length: 60567, dtype: object

In [261]:
# we can now append this data to the dataset, along with the coordinates: 
cr_reldata_f = pd.concat([cr_reldata, cr_servicedate], axis=1).drop(["service_date"], axis=1)
cr_reldata_f.groupby(["gtfs_route_id", "mod_service_date"]).mean()

gtfs_route_short_name  \
gtfs_route_id    mod_service_date                          
CR-Fairmount     weekday                             NaN   
                 weekend                             NaN   
CR-Fitchburg     weekday                             NaN   
                 weekend                             NaN   
CR-Franklin      weekday                             NaN   
                 weekend                             NaN   
CR-Greenbush     weekday                             NaN   
                 weekend                             NaN   
CR-Haverhill     weekday                             NaN   
                 weekend                             NaN   
CR-Kingston      weekday                             NaN   
                 weekend                             NaN   
CR-Lowell        weekday                             NaN   
                 weekend                             NaN   
CR-Middleborough weekday                             NaN   
                 weekend                             NaN   
CR-Needham       weekday                             NaN   
                 weekend                             NaN   
CR-Newburyport   weekday                             NaN   
                 weekend                             NaN   
CR-Providence    weekday                             NaN   
                 weekend                             NaN   
CR-Worcester     weekday                             NaN   
                 weekend                             NaN   

                                   gtfs_route_long_name  otp_numerator  \
gtfs_route_id    mod_service_date                                        
CR-Fairmount     weekday                            NaN      18.155758   
                 weekend                            NaN      14.818040   
CR-Fitchburg     weekday                            NaN      14.544545   
                 weekend                            NaN       6.246711   
CR-Franklin      weekday                            NaN      15.834242   
                 weekend                            NaN       7.307273   
CR-Greenbush     weekday                            NaN      10.902121   
                 weekend                            NaN       7.568027   
CR-Haverhill     weekday                            NaN      17.598788   
                 weekend                            NaN       5.661345   
CR-Kingston      weekday                            NaN      10.775152   
                 weekend                            NaN       7.584553   
CR-Lowell        weekday                            NaN      20.939964   
                 weekend                            NaN       7.467601   
CR-Middleborough weekday                            NaN      10.829091   
                 weekend                            NaN       7.509834   
CR-Needham       weekday                            NaN      14.045679   
                 weekend                            NaN       8.256623   
CR-Newburyport   weekday                            NaN      13.856631   
                 weekend                            NaN       8.428724   
CR-Providence    weekday                            NaN      15.099612   
                 weekend                            NaN       6.772727   
CR-Worcester     weekday                            NaN      20.093030   
                 weekend                            NaN       7.589450   

                                   otp_denominator  cancelled_numerator  \
gtfs_route_id    mod_service_date                                         
CR-Fairmount     weekday                 18.817879                  NaN   
                 weekend                 15.477449                  NaN   
CR-Fitchburg     weekday                 16.925758                  NaN   
                 weekend                  7.042763                  NaN   
CR-Franklin      weekday                 18.227879                  NaN   
                 weekend

In [262]:
cr_latlong#["route_id"].unique()

,X,Y,stop_name,route_id,daytype
0,-71.133246,42.238405,Readville,CR-Fairmount,weekday
1,-71.119270,42.253638,Fairmount,CR-Fairmount,weekday
2,-71.085475,42.280994,Morton Street,CR-Fairmount,weekday
3,-71.078140,42.292246,Talbot Avenue,CR-Fairmount,weekday
4,-71.076833,42.305037,Four Corners/Geneva,CR-Fairmount,weekday
...,...,...,...,...,...
9805,-71.277044,42.310370,Wellesley Hills,CR-Worcester,weekday
9806,-71.294173,42.297526,Wellesley Square,CR-Worcester,weekday
9807,-71.347133,42.285719,Natick Center,CR-Worcester,weekday
9808,-71.391797,42.283064,West Natick,CR-Worcester,weekday


In [263]:
cr_rel_f = []

for x in cr_latlong["X"].unique():

    for y in cr_latlong[cr_latlong["X"]==x]["Y"].unique():

        for route in cr_latlong[cr_latlong["X"]==x]["route_id"].unique():

            for daytype in ["weekday", "weekend"]:

                cr_rel_f.append(
                    [
                    x,
                    y,
                    cr_latlong[cr_latlong["X"] == x]["stop_name"].values[0],
                    route,
                    daytype,
                    cr_reldata_f[(cr_reldata_f["gtfs_route_id"]==route) & (cr_reldata_f["mod_service_date"]==daytype)]["reliability_metric"].mean()
                    ]
                    )
            

cr_rel_f = pd.DataFrame(cr_rel_f)
cr_rel_f.columns = ["X", "Y", "Stop", "Route", "Daytype", "Reliability_Metric"]
cr_rel_f


,X,Y,Stop,Route,Daytype,Reliability_Metric
0,-71.133246,42.238405,Readville,CR-Fairmount,weekday,0.961774
1,-71.133246,42.238405,Readville,CR-Fairmount,weekend,0.958481
2,-71.119270,42.253638,Fairmount,CR-Fairmount,weekday,0.961774
3,-71.119270,42.253638,Fairmount,CR-Fairmount,weekend,0.958481
4,-71.085475,42.280994,Morton Street,CR-Fairmount,weekday,0.961774
...,...,...,...,...,...,...
265,-71.647076,42.269644,Westborough,CR-Worcester,weekend,0.845773
266,-71.139883,42.357293,Boston Landing,CR-Worcester,weekday,0.837742
267,-71.139883,42.357293,Boston Landing,CR-Worcester,weekend,0.845773
268,-71.794888,42.261461,Worcester,CR-Worcester,weekday,0.837742


### RT-Ridership

In [264]:
rt_ridership_data # for this step, we essentially repeat all the code from the CR-Ridership section. 

,FID,mode,season,route_id,route_name,direction_id,day_type_id,day_type_name,time_period_id,time_period_name,stop_name,stop_id,total_ons,total_offs,number_service_days,average_ons,average_offs,average_flow
0,1,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Allston Street,place-alsgr,0,17,77,0,0,4
1,2,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Arlington,place-armnl,2675,8021,77,35,104,381
2,3,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Babcock Street,place-babck,0,151,77,0,2,8
3,4,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Back of the Hill,place-bckhl,0,36,77,0,0,4
4,5,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Beaconsfield,place-bcnfd,12,67,77,0,1,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7915,7916,1,Fall 2017,Red,Red Line,0,day_type_02,saturday,time_period_10,OFF_PEAK,Porter,place-portr,59366,9940,16,3710,621,13842
7916,7917,1,Fall 2017,Red,Red Line,0,day_type_02,saturday,time_period_10,OFF_PEAK,Quincy Adams,place-qamnl,388,26507,16,24,1657,1549
7917,7918,1,Fall 2017,Red,Red Line,0,day_type_02,saturday,time_period_10,OFF_PEAK,Quincy Center,place-qnctr,2128,67000,16,133,4188,3182
7918,7919,1,Fall 2017,Red,Red Line,0,day_type_02,saturday,time_period_10,OFF_PEAK,Savin Hill,place-shmnl,2292,18338,16,143,1146,7237


In [265]:
# change the day_type_name to be more uniform
rt_ridership_data["day_type_name"] = rt_ridership_data["day_type_name"].apply((lambda day: day if day =="weekday" else "weekend"))
rt_ridership_data

,FID,mode,season,route_id,route_name,direction_id,day_type_id,day_type_name,time_period_id,time_period_name,stop_name,stop_id,total_ons,total_offs,number_service_days,average_ons,average_offs,average_flow
0,1,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Allston Street,place-alsgr,0,17,77,0,0,4
1,2,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Arlington,place-armnl,2675,8021,77,35,104,381
2,3,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Babcock Street,place-babck,0,151,77,0,2,8
3,4,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Back of the Hill,place-bckhl,0,36,77,0,0,4
4,5,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Beaconsfield,place-bcnfd,12,67,77,0,1,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7915,7916,1,Fall 2017,Red,Red Line,0,day_type_02,weekend,time_period_10,OFF_PEAK,Porter,place-portr,59366,9940,16,3710,621,13842
7916,7917,1,Fall 2017,Red,Red Line,0,day_type_02,weekend,time_period_10,OFF_PEAK,Quincy Adams,place-qamnl,388,26507,16,24,1657,1549
7917,7918,1,Fall 2017,Red,Red Line,0,day_type_02,weekend,time_period_10,OFF_PEAK,Quincy Center,place-qnctr,2128,67000,16,133,4188,3182
7918,7919,1,Fall 2017,Red,Red Line,0,day_type_02,weekend,time_period_10,OFF_PEAK,Savin Hill,place-shmnl,2292,18338,16,143,1146,7237


In [266]:
# Unfortunately, the data has an issue with the Green line (my favorite rapid transit line, so we can't let this slide). 
# Specifically, all Green line data is marked with "Green", rather than "Green-B" to denote a specific route.
gl_help_data = pd.read_csv("MBTA_Rapid_Transit_Stop_Distances.csv")
gl_help_data

,route_id,origin,destination,from_station_id,from_station_name,to_station_id,to_station_name,distance_between_miles,cumulative_dist_miles,ObjectId
0,Red,Alewife,Ashmont,place-alfcl,Alewife,place-davis,Davis,0.960,0.960,1
1,Red,Alewife,Ashmont,place-davis,Davis,place-portr,Porter,0.660,1.620,2
2,Red,Alewife,Ashmont,place-portr,Porter,place-harsq,Harvard,0.990,2.610,3
3,Red,Alewife,Ashmont,place-harsq,Harvard,place-cntsq,Central,1.080,3.690,4
4,Red,Alewife,Ashmont,place-cntsq,Central,place-knncl,Kendall/MIT,0.950,4.640,5
...,...,...,...,...,...,...,...,...,...,...
227,Green C,North Station,Cleveland Circle,place-cool,Coolidge Corner,place-bcnwa,Washington Square,0.757,4.864,228
228,Green C,North Station,Cleveland Circle,place-bcnwa,Washington Square,place-clmnl,Cleveland Circle,0.755,5.619,229
229,Green C,Cleveland Circle,North Station,place-clmnl,Cleveland Circle,place-bcnwa,Washington Square,0.763,0.763,230
230,Green C,Cleveland Circle,North Station,place-bcnwa,Washington Square,place-cool,Coolidge Corner,0.749,1.512,231


In [267]:
rt_latlong = [] # we can now use the helpful data from the new dataset to associate Green stops with their specific route:
rtnonstops = []
for stop in rt_ridership_data["stop_name"].values:

    if stop in rtstopsdata["stop_name"].values and stop in gl_help_data["from_station_name"].values: # some of the ridership stops are not in the stops dataset... 

        rt_latlong.append(
            [
                rtstopsdata[rtstopsdata["stop_name"] == stop]["X"].values[0],
                rtstopsdata[rtstopsdata["stop_name"] == stop]["Y"].values[0],
                rtstopsdata[rtstopsdata["stop_name"] == stop]["stop_name"].values[0],
                gl_help_data[gl_help_data["from_station_name"] == stop]["route_id"].values[0],
                rt_ridership_data[rt_ridership_data["stop_name"] == stop]["day_type_name"].values[0]
            ]
            )

    else:
        rtnonstops.append(stop) # the ridership stops not included in the stops dataset: what are they? 

rt_latlong = pd.DataFrame(rt_latlong, columns=["X", "Y", "stop_name","route_id", "daytype"])
    
rt_latlong # looks great! 

,X,Y,stop_name,route_id,daytype
0,-71.070893,42.351902,Arlington,Green C,weekday
1,-71.140455,42.335765,Beaconsfield,Green D,weekday
2,-71.135330,42.339394,Washington Square,Green C,weekday
3,-71.100258,42.349293,Blandford Street,Green B,weekday
4,-71.064590,42.353020,Boylston,Green C,weekday
...,...,...,...,...,...
4945,-71.119149,42.388400,Porter,Red,weekday
4946,-71.007153,42.233391,Quincy Adams,Red,weekday
4947,-71.005409,42.251809,Quincy Center,Red,weekday
4948,-71.053331,42.311290,Savin Hill,Red,weekday


In [268]:
rt_ridership_data

,FID,mode,season,route_id,route_name,direction_id,day_type_id,day_type_name,time_period_id,time_period_name,stop_name,stop_id,total_ons,total_offs,number_service_days,average_ons,average_offs,average_flow
0,1,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Allston Street,place-alsgr,0,17,77,0,0,4
1,2,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Arlington,place-armnl,2675,8021,77,35,104,381
2,3,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Babcock Street,place-babck,0,151,77,0,2,8
3,4,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Back of the Hill,place-bckhl,0,36,77,0,0,4
4,5,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Beaconsfield,place-bcnfd,12,67,77,0,1,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7915,7916,1,Fall 2017,Red,Red Line,0,day_type_02,weekend,time_period_10,OFF_PEAK,Porter,place-portr,59366,9940,16,3710,621,13842
7916,7917,1,Fall 2017,Red,Red Line,0,day_type_02,weekend,time_period_10,OFF_PEAK,Quincy Adams,place-qamnl,388,26507,16,24,1657,1549
7917,7918,1,Fall 2017,Red,Red Line,0,day_type_02,weekend,time_period_10,OFF_PEAK,Quincy Center,place-qnctr,2128,67000,16,133,4188,3182
7918,7919,1,Fall 2017,Red,Red Line,0,day_type_02,weekend,time_period_10,OFF_PEAK,Savin Hill,place-shmnl,2292,18338,16,143,1146,7237


In [269]:
# now, we modify the ridership dataset so that it does not include stops which aren't listed in the stops dataset,
rt_ridership_f = rt_ridership_data[rt_ridership_data["stop_name"].isin(rt_latlong["stop_name"])][~rt_ridership_data["stop_name"].isin(pd.Series(rtnonstops).unique())] # nice
rt_ridership_f.index = rt_latlong.index

# we can then append the lat and lon: 
rt_ridership_f = pd.concat([rt_ridership_f,rt_latlong[["X", "Y"]]], axis=1)
rt_ridership_f[rt_ridership_f["average_flow"].notnull()]


<ipython-input-269-03af95c79386>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rt_ridership_f = rt_ridership_data[rt_ridership_data["stop_name"].isin(rt_latlong["stop_name"])][~rt_ridership_data["stop_name"].isin(pd.Series(rtnonstops).unique())] # nice


,FID,mode,season,route_id,route_name,direction_id,day_type_id,day_type_name,time_period_id,time_period_name,stop_name,stop_id,total_ons,total_offs,number_service_days,average_ons,average_offs,average_flow,X,Y
0,2,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Arlington,place-armnl,2675,8021,77,35,104,381,-71.070893,42.351902
1,5,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Beaconsfield,place-bcnfd,12,67,77,0,1,44,-71.140455,42.335765
2,6,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Washington Square,place-bcnwa,0,22,77,0,0,2,-71.135330,42.339394
3,7,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Blandford Street,place-bland,3,52,77,0,1,15,-71.100258,42.349293
4,9,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Boylston,place-boyls,1632,3604,77,21,47,450,-71.064590,42.353020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4945,7916,1,Fall 2017,Red,Red Line,0,day_type_02,weekend,time_period_10,OFF_PEAK,Porter,place-portr,59366,9940,16,3710,621,13842,-71.119149,42.388400
4946,7917,1,Fall 2017,Red,Red Line,0,day_type_02,weekend,time_period_10,OFF_PEAK,Quincy Adams,place-qamnl,388,26507,16,24,1657,1549,-71.007153,42.233391
4947,7918,1,Fall 2017,Red,Red Line,0,day_type_02,weekend,time_period_10,OFF_PEAK,Quincy Center,place-qnctr,2128,67000,16,133,4188,3182,-71.005409,42.251809
4948,7919,1,Fall 2017,Red,Red Line,0,day_type_02,weekend,time_period_10,OFF_PEAK,Savin Hill,place-shmnl,2292,18338,16,143,1146,7237,-71.053331,42.311290


In [270]:
rt_ridership_f#.groupby(["stop_name"]).mean()

,FID,mode,season,route_id,route_name,direction_id,day_type_id,day_type_name,time_period_id,time_period_name,stop_name,stop_id,total_ons,total_offs,number_service_days,average_ons,average_offs,average_flow,X,Y
0,2,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Arlington,place-armnl,2675,8021,77,35,104,381,-71.070893,42.351902
1,5,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Beaconsfield,place-bcnfd,12,67,77,0,1,44,-71.140455,42.335765
2,6,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Washington Square,place-bcnwa,0,22,77,0,0,2,-71.135330,42.339394
3,7,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Blandford Street,place-bland,3,52,77,0,1,15,-71.100258,42.349293
4,9,0,Fall 2019,Green,Green Line,0,day_type_01,weekday,time_period_01,VERY_EARLY_MORNING,Boylston,place-boyls,1632,3604,77,21,47,450,-71.064590,42.353020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4945,7916,1,Fall 2017,Red,Red Line,0,day_type_02,weekend,time_period_10,OFF_PEAK,Porter,place-portr,59366,9940,16,3710,621,13842,-71.119149,42.388400
4946,7917,1,Fall 2017,Red,Red Line,0,day_type_02,weekend,time_period_10,OFF_PEAK,Quincy Adams,place-qamnl,388,26507,16,24,1657,1549,-71.007153,42.233391
4947,7918,1,Fall 2017,Red,Red Line,0,day_type_02,weekend,time_period_10,OFF_PEAK,Quincy Center,place-qnctr,2128,67000,16,133,4188,3182,-71.005409,42.251809
4948,7919,1,Fall 2017,Red,Red Line,0,day_type_02,weekend,time_period_10,OFF_PEAK,Savin Hill,place-shmnl,2292,18338,16,143,1146,7237,-71.053331,42.311290


In [271]:
rt_ridership_f["day_type_name"].value_counts()

weekday    4050
weekend     900
Name: day_type_name, dtype: int64

In [272]:
rt_rid_f = [] # better organization: form a data table of [Lat, Long, Route name, and average daily load] for all routes.

for x in rt_latlong["X"].unique():

    for y in rt_latlong[rt_latlong["X"]==x]["Y"].unique():

        for route in rt_latlong[rt_latlong["X"]==x]["route_id"].unique():

            for day in rt_ridership_f[rt_ridership_f["X"]==x]["day_type_name"].unique():

                rt_rid_f.append(
                    [
                    x,
                    y,
                    rt_latlong[rt_latlong["X"]==x]["stop_name"].unique()[0],
                    route,
                    day,
                    rt_ridership_f[(rt_ridership_f["X"]==x) & (rt_ridership_f["day_type_name"]==day)]["average_flow"].mean()
                    ]
                    )
            

rt_rid_f = pd.DataFrame(rt_rid_f)
rt_rid_f.columns = ["X", "Y", "Stop", "Route", "day", "average_flow"]
rt_rid_f # good soup. 

,X,Y,Stop,Route,day,average_flow
0,-71.070893,42.351902,Arlington,Green C,weekday,4540.851852
1,-71.070893,42.351902,Arlington,Green C,weekend,26624.333333
2,-71.140455,42.335765,Beaconsfield,Green D,weekday,645.148148
3,-71.140455,42.335765,Beaconsfield,Green D,weekend,3291.250000
4,-71.135330,42.339394,Washington Square,Green C,weekday,164.092593
...,...,...,...,...,...,...
135,-71.020337,42.266514,Wollaston,Red,weekend,4282.416667
136,-71.142483,42.395428,Alewife,Red,weekday,670.888889
137,-71.142483,42.395428,Alewife,Red,weekend,2081.833333
138,-71.057655,42.330154,Andrew,Red,weekday,5247.888889


### RT-Reliability

In [273]:
# Again, we can go ahead and copy code from a previous section (this time, we borrow from CR-Reliability)
rt_reldata[rt_reldata["route_category"]=="Green Line"]

,service_date,gtfs_route_id,gtfs_route_short_name,gtfs_route_long_name,gtfs_route_desc,route_category,mode_type,peak_offpeak_ind,metric_type,otp_numerator,otp_denominator,cancelled_numerator,ObjectId,reliability_metric
4,2021/11/30 05:00:00+00,Green-E,NaN,NaN,Rapid Transit,Green Line,Rail,OFF_PEAK,Passenger Wait Time,17647.411728,22249.087595,NaN,5,0.793175
5,2021/11/30 05:00:00+00,Green-E,NaN,NaN,Rapid Transit,Green Line,Rail,PEAK,Passenger Wait Time,11559.097208,14843.362431,NaN,6,0.778738
6,2021/11/30 05:00:00+00,Green-D,NaN,NaN,Rapid Transit,Green Line,Rail,PEAK,Passenger Wait Time,13104.427461,17697.356870,NaN,7,0.740474
7,2021/11/30 05:00:00+00,Green-D,NaN,NaN,Rapid Transit,Green Line,Rail,OFF_PEAK,Passenger Wait Time,16295.696595,21099.846969,NaN,8,0.772313
8,2021/11/30 05:00:00+00,Green-C,NaN,NaN,Rapid Transit,Green Line,Rail,PEAK,Passenger Wait Time,7163.992177,9852.003149,NaN,9,0.727161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636524,2016/01/01 05:00:00+00,Green-B,NaN,NaN,Rapid Transit,Green Line,Rail,OFF_PEAK,Passenger Wait Time,1324.361000,1925.440000,NaN,636525,0.687823
636565,2016/01/02 05:00:00+00,Green-E,NaN,NaN,Rapid Transit,Green Line,Rail,OFF_PEAK,Passenger Wait Time,749.001100,983.531800,NaN,636566,0.761542
636566,2016/01/02 05:00:00+00,Green-D,NaN,NaN,Rapid Transit,Green Line,Rail,OFF_PEAK,Passenger Wait Time,1406.101000,1746.044000,NaN,636567,0.805307
636568,2016/01/02 05:00:00+00,Green-C,NaN,NaN,Rapid Transit,Green Line,Rail,OFF_PEAK,Passenger Wait Time,1162.634000,1611.579000,NaN,636569,0.721425


In [274]:
# modify dates
rt_servicedate = rt_reldata["service_date"].apply(lambda date: datetime.strptime(date[:10], '%Y/%m/%d').weekday())
rt_servicedate = rt_servicedate.apply(lambda date: "weekday" if date < 5 else "weekend")
rt_servicedate.name = "mod_service_date"
rt_servicedate # wa la

0         weekday
1         weekday
2         weekday
3         weekday
4         weekday
           ...   
683084    weekend
683085    weekend
683086    weekend
683087    weekend
683088    weekend
Name: mod_service_date, Length: 46720, dtype: object

In [275]:
rt_latlong

,X,Y,stop_name,route_id,daytype
0,-71.070893,42.351902,Arlington,Green C,weekday
1,-71.140455,42.335765,Beaconsfield,Green D,weekday
2,-71.135330,42.339394,Washington Square,Green C,weekday
3,-71.100258,42.349293,Blandford Street,Green B,weekday
4,-71.064590,42.353020,Boylston,Green C,weekday
...,...,...,...,...,...
4945,-71.119149,42.388400,Porter,Red,weekday
4946,-71.007153,42.233391,Quincy Adams,Red,weekday
4947,-71.005409,42.251809,Quincy Center,Red,weekday
4948,-71.053331,42.311290,Savin Hill,Red,weekday


In [276]:
# we can now append this data to the dataset, along with the coordinates: 
rt_reldata_f = pd.concat([rt_reldata, rt_servicedate], axis=1).drop(["service_date"], axis=1)
rt_reldata_f#.groupby(["gtfs_route_id", "mod_service_date"]).mean()

,gtfs_route_id,gtfs_route_short_name,gtfs_route_long_name,gtfs_route_desc,route_category,mode_type,peak_offpeak_ind,metric_type,otp_numerator,otp_denominator,cancelled_numerator,ObjectId,reliability_metric,mod_service_date
0,Red,NaN,NaN,Rapid Transit,Red Line,Rail,OFF_PEAK,Passenger Wait Time,116834.457868,131484.501852,NaN,1,0.888580,weekday
1,Red,NaN,NaN,Rapid Transit,Red Line,Rail,PEAK,Passenger Wait Time,103289.534378,117932.406159,NaN,2,0.875837,weekday
2,Orange,NaN,NaN,Rapid Transit,Orange Line,Rail,PEAK,Passenger Wait Time,76108.743163,81447.171476,NaN,3,0.934455,weekday
3,Orange,NaN,NaN,Rapid Transit,Orange Line,Rail,OFF_PEAK,Passenger Wait Time,89730.162237,97088.213286,NaN,4,0.924213,weekday
4,Green-E,NaN,NaN,Rapid Transit,Green Line,Rail,OFF_PEAK,Passenger Wait Time,17647.411728,22249.087595,NaN,5,0.793175,weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683084,751,NaN,NaN,Rapid Transit,Silver Line,Bus,OFF_PEAK,Headway / Schedule Adherence,1427.000000,1528.000000,NaN,683085,0.933901,weekend
683085,749,NaN,NaN,Rapid Transit,Silver Line,Bus,OFF_PEAK,Headway / Schedule Adherence,2230.000000,2712.000000,NaN,683086,0.822271,weekend
683086,746,NaN,NaN,Rapid Transit,Silver Line,Bus,OFF_PEAK,Headway / Schedule Adherence,56.000000,121.000000,NaN,683087,0.462810,weekend
683087,742,NaN,NaN,Rapid Transit,Silver Line,Bus,OFF_PEAK,Headway / Schedule Adherence,506.000000,752.000000,NaN,683088,0.672872,weekend


In [277]:
rt_latlong#["route_id"].unique()

,X,Y,stop_name,route_id,daytype
0,-71.070893,42.351902,Arlington,Green C,weekday
1,-71.140455,42.335765,Beaconsfield,Green D,weekday
2,-71.135330,42.339394,Washington Square,Green C,weekday
3,-71.100258,42.349293,Blandford Street,Green B,weekday
4,-71.064590,42.353020,Boylston,Green C,weekday
...,...,...,...,...,...
4945,-71.119149,42.388400,Porter,Red,weekday
4946,-71.007153,42.233391,Quincy Adams,Red,weekday
4947,-71.005409,42.251809,Quincy Center,Red,weekday
4948,-71.053331,42.311290,Savin Hill,Red,weekday


In [278]:
# generatee a final dataset
rt_rel_f = []

for x in rt_latlong["X"].unique():

    for y in rt_latlong[rt_latlong["X"]==x]["Y"].unique():

        for route in rt_latlong[rt_latlong["X"]==x]["route_id"].unique():
            route = route.replace(" ", "-")

            for daytype in ["weekday", "weekend"]:

                rt_rel_f.append(
                    [
                    x,
                    y,
                    rt_latlong[rt_latlong["X"]==x]["stop_name"].unique()[0],
                    route,
                    daytype,
                    rt_reldata_f[(rt_reldata_f["gtfs_route_id"]==route) & (rt_reldata_f["mod_service_date"]==daytype)]["reliability_metric"].mean()
                    ]
                    )
            

rt_rel_f = pd.DataFrame(rt_rel_f)
rt_rel_f.columns = ["X", "Y", "Stop", "Route", "Daytype", "Reliability_Metric"]
rt_rel_f

,X,Y,Stop,Route,Daytype,Reliability_Metric
0,-71.070893,42.351902,Arlington,Green-C,weekday,0.774933
1,-71.070893,42.351902,Arlington,Green-C,weekend,0.770911
2,-71.140455,42.335765,Beaconsfield,Green-D,weekday,0.772483
3,-71.140455,42.335765,Beaconsfield,Green-D,weekend,0.767135
4,-71.135330,42.339394,Washington Square,Green-C,weekday,0.774933
...,...,...,...,...,...,...
135,-71.020337,42.266514,Wollaston,Red,weekend,0.891200
136,-71.142483,42.395428,Alewife,Red,weekday,0.911357
137,-71.142483,42.395428,Alewife,Red,weekend,0.891200
138,-71.057655,42.330154,Andrew,Red,weekday,0.911357


### Final Data Modifications

Some column names like "Reliability_metric" are clunky. We can make these much more elegant: 

In [279]:
cr_rel_f.columns = ["X", "Y", "Stop", "Route", "Daytype", "Reliability Score"]
rt_rel_f.columns = ["X", "Y", "Stop", "Route", "Daytype", "Reliability Score"]

cr_rid_f.columns = ["X", "Y", "Stop", "Route", "day", "Average Load"]
rt_rid_f.columns = ["X", "Y", "Stop", "Route", "day", "Average Flow"] # the metric here is defined as "flow" for the dataset. I should have clarified this earlier. 

In [281]:
# for the rapid transit data in particular,  we can add an extra column describing the color
# hex codes from https://www.w3schools.com/colors/colors_picker.asp
def makecolor(route):
    if route == "Green E":# for the greens: give a different shade of green 
        return "003300"
    elif route == "Green D":
        return "006600"
    elif route == "Green C":
        return "33cc33"
    elif route == "Green B":
        return "00ff00"

    elif route == "Red":
        return "ff0000"
    elif route == "Blue":
        return "0066ff"
    else: #ornge 
        return "ff6600"

rt_rel_f_color = rt_rel_f["Route"].apply(makecolor)
rt_rel_f_color.name = "Color"
rt_rel_f = pd.concat([rt_rel_f, rt_rel_f_color], axis=1)

rt_rid_f_color = rt_rid_f["Route"].apply(makecolor)
rt_rid_f_color.name = "Color"
rt_rid_f = pd.concat([rt_rid_f, rt_rid_f_color], axis=1)


rt_rel_f

,X,Y,Stop,Route,Daytype,Reliability Score,Color
0,-71.070893,42.351902,Arlington,Green-C,weekday,0.774933,ff6600
1,-71.070893,42.351902,Arlington,Green-C,weekend,0.770911,ff6600
2,-71.140455,42.335765,Beaconsfield,Green-D,weekday,0.772483,ff6600
3,-71.140455,42.335765,Beaconsfield,Green-D,weekend,0.767135,ff6600
4,-71.135330,42.339394,Washington Square,Green-C,weekday,0.774933,ff6600
...,...,...,...,...,...,...,...
135,-71.020337,42.266514,Wollaston,Red,weekend,0.891200,ff0000
136,-71.142483,42.395428,Alewife,Red,weekday,0.911357,ff0000
137,-71.142483,42.395428,Alewife,Red,weekend,0.891200,ff0000
138,-71.057655,42.330154,Andrew,Red,weekday,0.911357,ff0000


In [282]:
# finally, split each dataset into weekend and weekday
cr_rel_f_weekday = cr_rel_f[cr_rel_f["Daytype"]=="weekday"]
cr_rel_f_weekend = cr_rel_f[cr_rel_f["Daytype"]=="weekend"]

cr_rid_f_weekday = cr_rid_f[cr_rid_f["day"]=="weekday"] # no weekend data

rt_rel_f_weekday = rt_rel_f[rt_rel_f["Daytype"]=="weekday"]
rt_rel_f_weekend = rt_rel_f[rt_rel_f["Daytype"]=="weekend"]

rt_rid_f_weekday = rt_rid_f[rt_rid_f["day"]=="weekday"]
rt_rid_f_weekend = rt_rid_f[rt_rid_f["day"]=="weekend"]

# ready to visualize! 

In [345]:
rt_rid_f_weekday

,X,Y,Stop,Route,day,Average Flow,Color
0,-71.070893,42.351902,Arlington,Green C,weekday,4540.851852,33cc33
2,-71.140455,42.335765,Beaconsfield,Green D,weekday,645.148148,006600
4,-71.135330,42.339394,Washington Square,Green C,weekday,164.092593,33cc33
6,-71.100258,42.349293,Blandford Street,Green B,weekday,844.185185,00ff00
8,-71.064590,42.353020,Boylston,Green C,weekday,4632.388889,33cc33
...,...,...,...,...,...,...,...
130,-71.065738,42.293126,Shawmut,Red,weekday,1069.425926,ff0000
132,-71.055242,42.352271,South Station,Red,weekday,6129.759259,ff0000
134,-71.020337,42.266514,Wollaston,Red,weekday,1959.537037,ff0000
136,-71.142483,42.395428,Alewife,Red,weekday,670.888889,ff0000


# Plot! 

### Commuter Rail Reliability Plot

Weekend

In [307]:
lines_horz = alt.Chart(cr_rel_f_weekend[cr_rel_f_weekend["Route"].isin(["CR-Haverhill", "CR-Lowell", "CR-Middelborough"])]
).mark_line(
    point=True,
    orient="horizontal"
).encode(
    alt.Y("Y:Q", 
          scale=alt.Scale(domain=[41.5,43]), 
          axis=alt.Axis(grid=False, title="Latitude")),
    alt.X('X:Q', 
          scale=alt.Scale(domain=[-72,-70.5]),
          axis=alt.Axis(grid=False, title="Longitude")),
    alt.Color("Reliability Score:Q")
).properties(
    width=600,
    height=600
).interactive(
)

lines_vert = alt.Chart(cr_rel_f_weekend[~cr_rel_f_weekend["Route"].isin(["CR-Haverhill", "CR-Lowell", "CR-Middelborough"])]
).mark_line(
    point=True,
    orient="vertical"
).encode(
    alt.Y("Y:Q", scale=alt.Scale(domain=[41.5,43])),
    alt.X('X:Q', scale=alt.Scale(domain=[-72,-70.5])),
    alt.Color("Reliability Score:Q")
).properties(
    width=600,
    height=600
).interactive(
)

points = alt.Chart(cr_rel_f_weekend
).mark_point(
    size=300
).encode(
    alt.X('X:Q', scale=alt.Scale(domain=[-72,-70.5])),
    alt.Y("Y:Q", scale=alt.Scale(domain=[41.5,43])),
    alt.Color("Reliability Score:Q", scale=alt.Scale(scheme = "purples")),
    alt.Tooltip(["Route:N", "Stop:N", "Reliability Score:Q"])
).properties(
    width=600,
    height=600,
    title = {
       "text" : "Average Reliability of MBTA Commuter Rail Routes on Weekends",
       "subtitle" : "Reliability is the number of on-time train observations over the number of total observations for a given day"
    }
).interactive(
)

lines_horz + lines_vert + points


alt.LayerChart(...)

weekday

In [355]:
lines_horz = alt.Chart(cr_rel_f_weekday[cr_rel_f_weekday["Route"].isin(["CR-Haverhill", "CR-Lowell", "CR-Middelborough"])]
).mark_line(
    point=True,
    orient="horizontal"
).encode(
    alt.Y("Y:Q", scale=alt.Scale(domain=[41.5,43])),
    alt.X('X:Q', scale=alt.Scale(domain=[-72,-70.5])),
    alt.Color("Reliability Score:Q")
).properties(
    width=600,
    height=600
).interactive(
)

lines_vert = alt.Chart(cr_rel_f_weekday[~cr_rel_f_weekday["Route"].isin(["CR-Haverhill", "CR-Lowell", "CR-Middelborough"])]
).mark_line(
    point=True,
    orient="vertical"
).encode(
    alt.Y("Y:Q", 
          scale=alt.Scale(domain=[41.5,43]),
          axis=alt.Axis(grid=False, title="Latitude")),
    alt.X('X:Q', 
          scale=alt.Scale(domain=[-72,-70.5]),
          axis=alt.Axis(grid=False, title="Longitude")),
    alt.Color("Reliability Score:Q")
).properties(
    width=600,
    height=600
).interactive(
)

points = alt.Chart(cr_rel_f_weekday
).mark_point(
    size=300
).encode(
    alt.X('X:Q', scale=alt.Scale(domain=[-72,-70.5])),
    alt.Y("Y:Q", scale=alt.Scale(domain=[41.5,43])),
    alt.Color("Reliability Score:Q", scale=alt.Scale(scheme = "purples")), # purple for official purple line
    alt.Tooltip(["Route:N", "Stop:N", "Reliability Score"])
).properties(
    width=600,
    height=600,
        title = {
       "text" : "Average Reliability of MBTA Commuter Rail Routes on Weekdays",
       "subtitle" : "Reliability is the number of on-time train observations over the number of total observations at a stop for a given day"
    }
).interactive(
)

lines_horz + lines_vert + points

alt.LayerChart(...)

### Commuter Rail Ridership Plot

weekday (no weekend data, so this is the only plot needed)

In [356]:
lines_horz = alt.Chart(cr_rid_f_weekday[cr_rid_f_weekday["Route"].isin(["CR-Haverhill", "CR-Lowell", "CR-Middelborough"])]
).mark_line(
    point=True,
    orient="horizontal"
).encode(
    alt.Y("Y:Q", 
          scale=alt.Scale(domain=[41.5,43]), 
          axis=alt.Axis(grid=False, title="Latitude")),
    alt.X('X:Q', 
          scale=alt.Scale(domain=[-72,-70.5]),
          axis=alt.Axis(grid=False, title="Longitude")),
    alt.Color("Average Load:Q")
).properties(
    width=600,
    height=600
).interactive(
)

lines_vert = alt.Chart(cr_rid_f_weekday[~cr_rid_f_weekday["Route"].isin(["CR-Haverhill", "CR-Lowell", "CR-Middelborough"])]
).mark_line(
    point=True,
    orient="vertical"
).encode(
    alt.Y("Y:Q", scale=alt.Scale(domain=[41.5,43])),
    alt.X('X:Q', scale=alt.Scale(domain=[-72,-70.5])),
    alt.Color("Average Load:Q")
).properties(
    width=600,
    height=600
).interactive(
)

points = alt.Chart(cr_rid_f_weekday
).mark_point(
    size=300
).encode(
    alt.X('X:Q', scale=alt.Scale(domain=[-72,-70.5])),
    alt.Y("Y:Q", scale=alt.Scale(domain=[41.5,43])),
    alt.Color("Average Load:Q", scale=alt.Scale(scheme = "purples")),
    alt.Tooltip(["Route:N", "Stop:N", "Average Load:Q"])
).properties(
    width=600,
    height=600,
        title = {
       "text" : "Average Load of MBTA Commuter Rail Routes on Weekdays",
       "subtitle" : "Load is the number of passengers on a train leaving a stop"
    }
).interactive(
)

lines_horz + lines_vert + points

alt.LayerChart(...)

### Rapid Transit Reliability Plot

weekday

In [339]:
lines_horz = alt.Chart(rt_rel_f_weekday[rt_rel_f_weekday["Route"].isin(["Green-C","Green-D", "Green-B", "Green-E", "Blue"])]
).mark_line(
    point=True,
    orient="vertical"
).encode(
    alt.Y("Y:Q", 
          scale=alt.Scale(domain=[42.18,42.5]), 
          axis=alt.Axis(grid=False, title="Latitude")),
    alt.X('X:Q', 
          scale=alt.Scale(domain=[-71.35,-70.9]),
          axis=alt.Axis(grid=False, title="Longitude")),
    alt.Color("Route", scale=alt.Scale(
        domain=['Green-C', 'Green-D', 'Green-B', 'Green-E', "Blue", "Red", "Orange"],
        range=["#33cc33", "#006600", "#00ff00", "#003300",  "#0066ff","#ff0000", "#ff6600"]
        ))
).properties(
    width=600,
    height=600
).interactive(
)

lines_vert= alt.Chart(rt_rel_f_weekday[~rt_rel_f_weekday["Route"].isin(["Green-C","Green-D", "Green-B", "Green-E", "Blue"])]
).mark_line(
    point=True,
    orient="horizontal"
).encode(
    alt.Y("Y:Q", 
          scale=alt.Scale(domain=[42.18,42.5]), 
          axis=alt.Axis(grid=False)),
    alt.X('X:Q', 
          scale=alt.Scale(domain=[-71.35,-70.9]),
          axis=alt.Axis(grid=False)),
    alt.Color("Route", scale=alt.Scale(
        domain = ["Red", "Orange"],
        range = ["#6600cc", "#ffffff"]
    ))
).properties(
    width=600,
    height=600
).interactive(
)

points = alt.Chart(rt_rel_f_weekday
).mark_point(
).encode(
    alt.X('X:Q', scale=alt.Scale(domain=[-71.35,-70.9])),
    alt.Y("Y:Q", scale=alt.Scale(domain=[42.18,42.5])),
    alt.Color("Route"),
    alt.Tooltip(["Route:N", "Stop:N", "Reliability Score:Q"]),
    alt.Size("Reliability Score:Q")
).properties(
    width=600,
    height=600,
        title = {
       "text" : "Average Reliability of MBTA Rapid Transit Routes on Weekdays",
       "subtitle" : "Reliability is the number of on-time train observations over the number of total observations for a given day"
    }
).interactive(
)

lines_horz + lines_vert + points

alt.LayerChart(...)

weekend

In [342]:
lines_horz = alt.Chart(rt_rel_f_weekend[rt_rel_f_weekend["Route"].isin(["Green-C","Green-D", "Green-B", "Green-E", "Blue"])]
).mark_line(
    point=True,
    orient="vertical"
).encode(
    alt.Y("Y:Q", 
          scale=alt.Scale(domain=[42.18,42.5]), 
          axis=alt.Axis(grid=False, title="Latitude")),
    alt.X('X:Q', 
          scale=alt.Scale(domain=[-71.35,-70.9]),
          axis=alt.Axis(grid=False, title="Longitude")),
    alt.Color("Route", scale=alt.Scale(
        domain=['Green-C', 'Green-D', 'Green-B', 'Green-E', "Blue", "Red", "Orange"],
        range=["#33cc33", "#006600", "#00ff00", "#003300",  "#0066ff","#ff0000", "#ff6600"]
        ))
).properties(
    width=600,
    height=600
).interactive(
)

lines_vert= alt.Chart(rt_rel_f_weekend[~rt_rel_f_weekend["Route"].isin(["Green-C","Green-D", "Green-B", "Green-E", "Blue"])]
).mark_line(
    point=True,
    orient="horizontal"
).encode(
    alt.Y("Y:Q", 
          scale=alt.Scale(domain=[42.18,42.5]), 
          axis=alt.Axis(grid=False)),
    alt.X('X:Q', 
          scale=alt.Scale(domain=[-71.35,-70.9]),
          axis=alt.Axis(grid=False)),
    alt.Color("Route", scale=alt.Scale(
        domain = ["Red", "Orange"],
        range = ["#6600cc", "#ffffff"]
    ))
).properties(
    width=600,
    height=600
).interactive(
)

points = alt.Chart(rt_rel_f_weekend
).mark_point(
).encode(
    alt.X('X:Q', scale=alt.Scale(domain=[-71.35,-70.9])),
    alt.Y("Y:Q", scale=alt.Scale(domain=[42.18,42.5])),
    alt.Color("Route"),
    alt.Tooltip(["Route:N", "Stop:N", "Reliability Score:Q"]),
    alt.Size("Reliability Score:Q")
).properties(
    width=600,
    height=600,
        title = {
       "text" : "Average Reliability of MBTA Rapid Transit Routes on Weekends",
       "subtitle" : "Reliability is the number of on-time train observations over the number of total observations for a given day"
    }
).interactive(
)

lines_horz + lines_vert + points

alt.LayerChart(...)

### Rapid Transit Ridership Plot

Weekday

In [351]:
lines_horz = alt.Chart(rt_rid_f_weekday[rt_rid_f_weekday["Route"].isin(["Green C","Green D", "Green B", "Green E", "Blue"])]
).mark_line(
    point=True,
    orient="vertical"
).encode(
    alt.Y("Y:Q", 
          scale=alt.Scale(domain=[42.18,42.5]), 
          axis=alt.Axis(grid=False, title="Latitude")),
    alt.X('X:Q', 
          scale=alt.Scale(domain=[-71.35,-70.9]),
          axis=alt.Axis(grid=False, title="Longitude")),
    alt.Color("Route", scale=alt.Scale(
        domain=['Green C', 'Green D', 'Green B', 'Green E', "Blue", "Red", "Orange"],
        range=["#33cc33", "#006600", "#00ff00", "#003300",  "#0066ff","#ff0000", "#ff6600"]
        ))
).properties(
    width=600,
    height=600
).interactive(
)

lines_vert= alt.Chart(rt_rid_f_weekday[~rt_rid_f_weekday["Route"].isin(["Green C","Green D", "Green B", "Green E", "Blue"])]
).mark_line(
    point=True,
    orient="horizontal"
).encode(
    alt.Y("Y:Q", 
          scale=alt.Scale(domain=[42.18,42.5]), 
          axis=alt.Axis(grid=False)),
    alt.X('X:Q', 
          scale=alt.Scale(domain=[-71.35,-70.9]),
          axis=alt.Axis(grid=False)),
    alt.Color("Route", scale=alt.Scale(
        domain = ["Red", "Orange"],
        range = ["#6600cc", "#ffffff"]
    ))
).properties(
    width=600,
    height=600
).interactive(
)

points = alt.Chart(rt_rid_f_weekday
).mark_point(
).encode(
    alt.X('X:Q', scale=alt.Scale(domain=[-71.35,-70.9])),
    alt.Y("Y:Q", scale=alt.Scale(domain=[42.18,42.5])),
    alt.Color("Route"),
    alt.Tooltip(["Route:N", "Stop:N", "Average Flow:Q"]),
    alt.Size("Average Flow:Q")
).properties(
    width=600,
    height=600,
        title = {
       "text" : "Average Flow of MBTA Rapid Transit Routes on Weekdays",
       "subtitle" : "Flow is the total number of passengers traveling between stations on a given day"
    }
).interactive(
)

lines_horz + lines_vert + points

alt.LayerChart(...)

Weekend

In [353]:
lines_horz = alt.Chart(rt_rid_f_weekend[rt_rid_f_weekend["Route"].isin(["Green C","Green D", "Green B", "Green E", "Blue"])]
).mark_line(
    point=True,
    orient="vertical"
).encode(
    alt.Y("Y:Q", 
          scale=alt.Scale(domain=[42.18,42.5]), 
          axis=alt.Axis(grid=False, title="Latitude")),
    alt.X('X:Q', 
          scale=alt.Scale(domain=[-71.35,-70.9]),
          axis=alt.Axis(grid=False, title="Longitude")),
    alt.Color("Route", scale=alt.Scale(
        domain=['Green C', 'Green D', 'Green B', 'Green E', "Blue", "Red", "Orange"],
        range=["#33cc33", "#006600", "#00ff00", "#003300",  "#0066ff","#ff0000", "#ff6600"]
        ))
).properties(
    width=600,
    height=600
).interactive(
)

lines_vert= alt.Chart(rt_rid_f_weekend[~rt_rid_f_weekend["Route"].isin(["Green C","Green D", "Green B", "Green E", "Blue"])]
).mark_line(
    point=True,
    orient="horizontal"
).encode(
    alt.Y("Y:Q", 
          scale=alt.Scale(domain=[42.18,42.5]), 
          axis=alt.Axis(grid=False)),
    alt.X('X:Q', 
          scale=alt.Scale(domain=[-71.35,-70.9]),
          axis=alt.Axis(grid=False)),
    alt.Color("Route", scale=alt.Scale(
        domain = ["Red", "Orange"],
        range = ["#6600cc", "#ffffff"]
    ))
).properties(
    width=600,
    height=600
).interactive(
)

points = alt.Chart(rt_rid_f_weekend
).mark_point(
).encode(
    alt.X('X:Q', scale=alt.Scale(domain=[-71.35,-70.9])),
    alt.Y("Y:Q", scale=alt.Scale(domain=[42.18,42.5])),
    alt.Color("Route"),
    alt.Tooltip(["Route:N", "Stop:N", "Average Flow:Q"]),
    alt.Size("Average Flow:Q")
).properties(
    width=600,
    height=600,
        title = {
       "text" : "Average Flow of MBTA Rapid Transit Routes on Weekends",
       "subtitle" : "Flow is the total number of passengers traveling between stations on a given day"
    }
).interactive(
)

lines_horz + lines_vert + points

alt.LayerChart(...)

## Final Remarks

Overall, I am satisfied with the visualizations- they look fairly nice, and depict all information accurately. I am especially happy with the latitude and longitude, as those allowed me to create visualizations where the points (stops, in this case) are situated in locations that are representative of their location in real life. 

__There are a couple of things I would have liked to improve upon__: 

* As you can see in the graphs, some of the lines which connect the stations (a visual representation of train routes) are jagged and unpleasant to the eye. This occurs when station "a" is further away from station "b" than station "c" iin actuality, but station "a" has a closer longitude or latitude and is connected to station "b" via a line on the graph, whereas station "c" should have been connected to station "b". I attempted to fix this by grouping the stations into vertical and horizontal groups (lines_horz and lines_vert), but this only solved part of the problem. 

* To make my visualizations nicer, I would have liked to have a map of Massachusetts (with county / town borders clearly defined) laid over each graph. However, even after hours of inspecting documentation, I could not find any feasible ways of doing this. Please let me know if you know of a simple way! 



## Links to Data

commuter rail stations: https://mbta-massdot.opendata.arcgis.com/datasets/commuter-rail-stations/explore?location=42.111974%2C-71.166936%2C8.71

rapid transit stations: https://mbta-massdot.opendata.arcgis.com/datasets/rapid-transit-stops/explore

bus stops: https://mbta-massdot.opendata.arcgis.com/maps/pati-bus-stops/about

reliability: https://mbta-massdot.opendata.arcgis.com/datasets/mbta-bus-commuter-rail-rapid-transit-reliability/explore

commuter rail ridership: https://mbta-massdot.opendata.arcgis.com/datasets/mbta-commuter-rail-ridership-by-trip-season-route-line-and-stop/explore

rapid transit ridership: https://mbta-massdot.opendata.arcgis.com/datasets/mbta-rail-ridership-by-time-period-season-route-line-and-stop/explore
